# Find duplicate questions on StackOverflow by their embeddings

In this assignment you will learn how to calculate a similarity for pieces of text. Using this approach you will know how to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

In this task you will you will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [1]:
import sys
sys.path.append("..")
from common.download_utils import download_week3_resources

download_week3_resources()

File data/train.tsv is already downloaded.
File data/validation.tsv is already downloaded.
File data/test.tsv is already downloaded.
File data/test_embeddings.tsv is already downloaded.


### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [2]:
from grader import Grader

In [3]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. You need to download it by following this [link](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing).
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [4]:
import gensim

In [7]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(
    './GoogleNews-vectors-negative300.bin',
    binary=True)
######### YOUR CODE HERE #############

### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [8]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [9]:
print(check_embeddings(wv_embeddings))

These embeddings look good.


## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above. This function should work with the input text as is without any preprocessing.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [10]:
import numpy as np

In [24]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    emb_list = []
    for word in question.split():
        if word in embeddings:
            #print(word)
            emb_list.append(embeddings[word])
            
    if len(emb_list):
        return np.mean(emb_list,axis=0)
    return np.zeros(dim)
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

To check the basic correctness of your implementation, run the function *question_to_vec_tests*.

In [25]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [26]:
print(question_to_vec_tests())

Basic tests are passed.


You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [28]:
from util import array_to_string

In [29]:
question2vec_result = []
for question in open('data/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

Current answer for task Question2Vec is: 0.01929389126598835
-0.02872721292078495
0.0460561104118824
0.0852593332529068
0.0243055559694767
-0...


Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [30]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for current ranking
    """
    return 1/len(dup_ranks)*np.sum([x<=k for x in dup_ranks])
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

Test your code on the tiny examples:

In [31]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [32]:
print(test_hits())

Basic test are passed.


In [33]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for current ranking
    """
    return 1/len(dup_ranks)*np.sum([(x<=k)/np.log2(1+x) for x in dup_ranks])
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

In [34]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [35]:
print(test_dcg())

Basic test are passed.


Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [36]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [37]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

Current answer for task HitsCount is: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.30000000000000004
0.4
0.5
0....


In [38]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

Current answer for task DCGScore is: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...


##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should upload *validation* corpus to evaluate current solution.

In [39]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [40]:
validation = read_corpus('data/validation.tsv')######### YOUR CODE HERE #############

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [52]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    v1 = question_to_vec(question, embeddings, dim)
    v2 = np.array([question_to_vec(x, embeddings, dim) for x in candidates])
    sim = cosine_similarity(v1.reshape(1,-1),v2)
    return [(x, candidates[x]) for x in np.argsort(-np.squeeze(sim))]
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

Test your code on the tiny examples:

In [53]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [54]:
print(test_rank_candidates())

Basic tests are passed.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [56]:
from tqdm import tqdm
wv_ranking = []
for line in tqdm(validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 10000/10000 [04:34<00:00, 36.43it/s]


In [57]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.217 | Hits@   1: 0.217
DCG@   5: 0.271 | Hits@   5: 0.319
DCG@  10: 0.288 | Hits@  10: 0.370
DCG@ 100: 0.326 | Hits@ 100: 0.560
DCG@ 500: 0.358 | Hits@ 500: 0.818
DCG@1000: 0.378 | Hits@1000: 1.000


If you did all the steps correctly, you should be frustrated by the received results. Let's try to understand why the quality is so low. First of all, when you work with some data it is necessary to have an idea how the data looks like. Print several questions from the data:

In [58]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


As you can see, we deal with the raw data. It means that we have many punctuation marks, special characters and unlowercased letters. In our case, it could lead to the situation where we can't find some embeddings, e.g. for the word "grid?". 

To solve this problem you should use the functions *text_prepare* from the previous assignments to prepare the data.

In [59]:
from util import text_prepare

Now transform all the questions from the validation set:

In [64]:
prepared_validation = []
for line in tqdm(validation):
    prepared_validation.append([text_prepare(x) for x in line])
    ######### YOUR CODE HERE #############

100%|██████████| 10000/10000 [00:26<00:00, 380.75it/s]


Let's evaluate the approach again after the preparation:

In [66]:
wv_prepared_ranking = []
for line in tqdm(prepared_validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 10000/10000 [03:59<00:00, 41.84it/s]


In [67]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

DCG@   1: 0.320 | Hits@   1: 0.320
DCG@   5: 0.387 | Hits@   5: 0.448
DCG@  10: 0.404 | Hits@  10: 0.500
DCG@ 100: 0.438 | Hits@ 100: 0.669
DCG@ 500: 0.460 | Hits@ 500: 0.844
DCG@1000: 0.477 | Hits@1000: 1.000


Now, prepare also train and test data, because you will need it in the future:

In [68]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [69]:
prepare_file('data/train.tsv','data/train_prepared.tsv')
prepare_file('data/test.tsv','data/test_prepared.tsv')

######################################
######### YOUR CODE HERE #############
######################################

**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [70]:
from util import matrix_to_string

In [71]:
w2v_ranks_results = []
prepared_test_data = 'data/test_prepared.tsv' ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

Current answer for task W2VTokenizedRanks is: 95	94	7	9	64	37	32	93	24	100	98	17	60	6	97	49	70	38	42	96	30	21	2	65	67	45	27	26	57	62	11	88	56	66	7...


## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace cannot be run on Windows and we recommend to use provided
[docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a number of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour.

In [73]:
! ./Starspace/starspace train -trainFile data/train_prepared.tsv -model starspace_embedding -trainMode 3 -minCount 2 -verbose true -fileFormat labelDoc -negSearchLimit 10 -lr 0.05
######### TRAINING HAPPENING HERE #############

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
Start to initialize starspace model.
Build dict from input file : data/train_prepared.tsv
Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Loading data from file : data/train_prepared.tsv
Total number of examples loaded : 999740
Initialized model weights. Model size :
matrix : 95058 100
Training epoch 0: 0.05 0.01


Epoch: 64.1%  lr: 0.043414  loss: 0.010584  eta: 0h4m  tot: 0h0m40s  (12.8%)%  lr: 0.050000  loss: 0.078742  eta: 0h5m  tot: 0h0m0s  (0.1%)1.1%  lr: 0.049940  loss: 0.051382  eta: 0h5m  tot: 0h0m0s  (0.2%)%  lr: 0.049930  loss: 0.048859  eta: 0h5m  tot: 0h0m0s  (0.3%)1.4%  lr: 0.049920  loss: 0.047498  eta: 0h5m  tot: 0h0m0s  (0.3%)2.2%  lr: 0.049840  loss: 0.040855  eta: 0h5m  tot: 0h0m1s  (0.4%)%  lr: 0.049840  loss: 0.040457  eta: 0h5m  tot: 0h0m1s  (0.5%)2.4%  lr: 0.049830  loss: 0.039841  eta: 0h5m  tot: 0h0m1s  (0.5%)3.5%  lr: 0.049720  loss: 0.034075  eta: 0h5m  tot: 0h0m2s  (0.7%)3.9%  lr: 0.049610  loss: 0.032561  eta: 0h5m  tot: 0h0m2s  (0.8%)4.9%  lr: 0.049510  loss: 0.030091  eta: 0h5m  tot: 0h0m3s  (1.0%)5.1%  lr: 0.049439  loss: 0.029476  eta: 0h5m  tot: 0h0m3s  (1.0%)5.3%  lr: 0.049419  loss: 0.028986  eta: 0h5m  tot: 0h0m3s  (1.1%)%  lr: 0.049399  loss: 0.027870  eta: 0h5m  tot: 0h0m4s  (1.1%)6.2%  lr: 0.049329  loss: 0.026935  eta: 0h5m  tot: 0h0m4s  (1.2%)%  lr: 0.049

Epoch: 100.0%  lr: 0.040000  loss: 0.008905  eta: h4m  tot: 0h1m1s  (20.0%))64.9%  lr: 0.043353  loss: 0.010533  eta: 0h4m  tot: 0h0m41s  (13.0%)65.2%  lr: 0.043323  loss: 0.010515  eta: 0h4m  tot: 0h0m41s  (13.0%)65.9%  lr: 0.043243  loss: 0.010475  eta: 0h4m  tot: 0h0m41s  (13.2%)66.7%  lr: 0.043173  loss: 0.010417  eta: 0h4m  tot: 0h0m42s  (13.3%)67.2%  lr: 0.043123  loss: 0.010387  eta: 0h4m  tot: 0h0m42s  (13.4%)69.1%  lr: 0.043013  loss: 0.010282  eta: 0h4m  tot: 0h0m43s  (13.8%)69.9%  lr: 0.042963  loss: 0.010239  eta: 0h4m  tot: 0h0m43s  (14.0%)70.9%  lr: 0.042873  loss: 0.010188  eta: 0h4m  tot: 0h0m44s  (14.2%)71.2%  lr: 0.042853  loss: 0.010167  eta: 0h4m  tot: 0h0m44s  (14.2%)%  lr: 0.042733  loss: 0.010111  eta: 0h4m  tot: 0h0m45s  (14.5%)72.9%  lr: 0.042683  loss: 0.010071  eta: 0h4m  tot: 0h0m45s  (14.6%)73.2%  lr: 0.042643  loss: 0.010060  eta: 0h4m  tot: 0h0m45s  (14.6%)%  lr: 0.042633  loss: 0.010041  eta: 0h4m  tot: 0h0m45s  (14.7%)75.2%  lr: 0.042493  loss: 0.009959

Epoch: 74.5%  lr: 0.032122  loss: 0.002696  eta: 0h3m  tot: 0h1m46s  (34.9%)3%  lr: 0.039830  loss: 0.002015  eta: 0h4m  tot: 0h1m1s  (20.3%)1.9%  lr: 0.039810  loss: 0.002334  eta: 0h3m  tot: 0h1m2s  (20.4%)2.0%  lr: 0.039790  loss: 0.002318  eta: 0h3m  tot: 0h1m2s  (20.4%)2.3%  lr: 0.039770  loss: 0.002429  eta: 0h3m  tot: 0h1m2s  (20.5%)3.2%  lr: 0.039700  loss: 0.002460  eta: 0h3m  tot: 0h1m2s  (20.6%)3.3%  lr: 0.039690  loss: 0.002461  eta: 0h3m  tot: 0h1m2s  (20.7%)4.4%  lr: 0.039550  loss: 0.002456  eta: 0h3m  tot: 0h1m3s  (20.9%)4.5%  lr: 0.039550  loss: 0.002445  eta: 0h3m  tot: 0h1m3s  (20.9%)4.8%  lr: 0.039540  loss: 0.002399  eta: 0h3m  tot: 0h1m3s  (21.0%)7.7%  lr: 0.039289  loss: 0.002508  eta: 0h3m  tot: 0h1m5s  (21.5%)8.6%  lr: 0.039219  loss: 0.002520  eta: 0h3m  tot: 0h1m5s  (21.7%)9.0%  lr: 0.039169  loss: 0.002537  eta: 0h3m  tot: 0h1m6s  (21.8%)9.5%  lr: 0.039129  loss: 0.002529  eta: 0h3m  tot: 0h1m6s  (21.9%)11.8%  lr: 0.038899  loss: 0.002545  eta: 0h3m  tot: 0h

Epoch: 100.0%  lr: 0.030000  loss: 0.002647  eta: 0h2m  tot: 0h1m59s  (40.0%)5.3%  lr: 0.032032  loss: 0.002687  eta: 0h3m  tot: 0h1m46s  (35.1%)75.6%  lr: 0.032002  loss: 0.002688  eta: 0h3m  tot: 0h1m46s  (35.1%)76.2%  lr: 0.031972  loss: 0.002685  eta: 0h3m  tot: 0h1m46s  (35.2%)76.4%  lr: 0.031942  loss: 0.002684  eta: 0h3m  tot: 0h1m47s  (35.3%)77.2%  lr: 0.031872  loss: 0.002679  eta: 0h3m  tot: 0h1m47s  (35.4%)77.4%  lr: 0.031872  loss: 0.002680  eta: 0h3m  tot: 0h1m47s  (35.5%)77.7%  lr: 0.031832  loss: 0.002675  eta: 0h3m  tot: 0h1m47s  (35.5%)78.1%  lr: 0.031742  loss: 0.002677  eta: 0h3m  tot: 0h1m48s  (35.6%)78.3%  lr: 0.031712  loss: 0.002678  eta: 0h3m  tot: 0h1m48s  (35.7%)78.6%  lr: 0.031682  loss: 0.002676  eta: 0h3m  tot: 0h1m48s  (35.7%)79.6%  lr: 0.031562  loss: 0.002678  eta: 0h3m  tot: 0h1m49s  (35.9%)79.9%  lr: 0.031542  loss: 0.002679  eta: 0h3m  tot: 0h1m49s  (36.0%)80.6%  lr: 0.031502  loss: 0.002674  eta: 0h3m  tot: 0h1m49s  (36.1%)%  lr: 0.031502  loss: 0.00

...

Epoch: 100.0%  lr: 0.020000  loss: 0.001919  eta: 0h1m  tot: 0h2m57s  (60.0%)3.4%  lr: 0.023183  loss: 0.001911  eta: 0h2m  tot: 0h2m38s  (52.7%)%  lr: 0.023173  loss: 0.001910  eta: 0h2m  tot: 0h2m39s  (52.8%)64.1%  lr: 0.023153  loss: 0.001907  eta: 0h2m  tot: 0h2m39s  (52.8%)64.7%  lr: 0.023073  loss: 0.001908  eta: 0h2m  tot: 0h2m39s  (52.9%)65.0%  lr: 0.023033  loss: 0.001908  eta: 0h2m  tot: 0h2m39s  (53.0%)65.6%  lr: 0.022983  loss: 0.001907  eta: 0h2m  tot: 0h2m40s  (53.1%)65.9%  lr: 0.022953  loss: 0.001910  eta: 0h2m  tot: 0h2m40s  (53.2%)%  lr: 0.022893  loss: 0.001912  eta: 0h2m  tot: 0h2m40s  (53.4%)67.6%  lr: 0.022843  loss: 0.001909  eta: 0h2m  tot: 0h2m41s  (53.5%)67.8%  lr: 0.022813  loss: 0.001909  eta: 0h2m  tot: 0h2m41s  (53.6%)68.0%  lr: 0.022773  loss: 0.001906  eta: 0h2m  tot: 0h2m41s  (53.6%)68.1%  lr: 0.022763  loss: 0.001905  eta: 0h2m  tot: 0h2m41s  (53.6%)%  lr: 0.022713  loss: 0.001906  eta: 0h2m  tot: 0h2m41s  (53.8%)69.4%  lr: 0.022693  loss: 0.001907  et

Epoch: 66.4%  lr: 0.012673  loss: 0.001589  eta: 0h1m  tot: 0h3m39s  (73.3%).2%  lr: 0.019990  loss: 0.000927  eta: 0h1m  tot: 0h2m58s  (60.0%)0.3%  lr: 0.019990  loss: 0.001034  eta: 0h1m  tot: 0h2m58s  (60.1%)0.4%  lr: 0.019960  loss: 0.001410  eta: 0h2m  tot: 0h2m58s  (60.1%)0.6%  lr: 0.019950  loss: 0.001632  eta: 0h2m  tot: 0h2m58s  (60.1%)%  lr: 0.019940  loss: 0.001585  eta: 0h2m  tot: 0h2m58s  (60.1%)m58s  (60.2%)1.2%  lr: 0.019890  loss: 0.001500  eta: 0h2m  tot: 0h2m58s  (60.2%)1.4%  lr: 0.019860  loss: 0.001565  eta: 0h2m  tot: 0h2m59s  (60.3%)%  lr: 0.019850  loss: 0.001574  eta: 0h2m  tot: 0h2m59s  (60.3%)3.5%  lr: 0.019720  loss: 0.001639  eta: 0h2m  tot: 0h3m0s  (60.7%)3.9%  lr: 0.019720  loss: 0.001657  eta: 0h2m  tot: 0h3m0s  (60.8%)4.1%  lr: 0.019680  loss: 0.001648  eta: 0h2m  tot: 0h3m0s  (60.8%)%  lr: 0.019550  loss: 0.001601  eta: 0h2m  tot: 0h3m1s  (61.1%)6.7%  lr: 0.019389  loss: 0.001576  eta: 0h2m  tot: 0h3m2s  (61.3%)7.9%  lr: 0.019289  loss: 0.001574  eta: 0

Epoch: 100.0%  lr: 0.010000  loss: 0.001557  eta: <1min   tot: 0h3m57s  (80.0%)7%  lr: 0.012473  loss: 0.001594  eta: 0h1m  tot: 0h3m40s  (73.5%)68.1%  lr: 0.012393  loss: 0.001592  eta: 0h1m  tot: 0h3m40s  (73.6%)68.3%  lr: 0.012362  loss: 0.001592  eta: 0h1m  tot: 0h3m40s  (73.7%)68.6%  lr: 0.012332  loss: 0.001592  eta: 0h1m  tot: 0h3m41s  (73.7%)69.2%  lr: 0.012292  loss: 0.001594  eta: 0h1m  tot: 0h3m41s  (73.8%)69.4%  lr: 0.012262  loss: 0.001593  eta: 0h1m  tot: 0h3m41s  (73.9%)%  lr: 0.012242  loss: 0.001594  eta: 0h1m  tot: 0h3m41s  (73.9%)69.7%  lr: 0.012222  loss: 0.001592  eta: 0h1m  tot: 0h3m41s  (73.9%)70.0%  lr: 0.012162  loss: 0.001592  eta: 0h1m  tot: 0h3m41s  (74.0%)%  lr: 0.012132  loss: 0.001592  eta: 0h1m  tot: 0h3m42s  (74.0%)71.0%  lr: 0.012062  loss: 0.001597  eta: 0h1m  tot: 0h3m42s  (74.2%)71.1%  lr: 0.012052  loss: 0.001596  eta: 0h1m  tot: 0h3m42s  (74.2%)71.6%  lr: 0.012022  loss: 0.001595  eta: 0h1m  tot: 0h3m42s  (74.3%)71.8%  lr: 0.012012  loss: 0.001593

Epoch: 68.9%  lr: 0.002693  loss: 0.001386  eta: <1min   tot: 0h4m38s  (93.8%).8%  lr: 0.009940  loss: 0.001720  eta: <1min   tot: 0h3m57s  (80.2%)1.1%  lr: 0.009890  loss: 0.001546  eta: <1min   tot: 0h3m57s  (80.2%)2.1%  lr: 0.009800  loss: 0.001509  eta: 0h1m  tot: 0h3m58s  (80.4%)3.1%  lr: 0.009700  loss: 0.001388  eta: <1min   tot: 0h3m58s  (80.6%)3.3%  lr: 0.009670  loss: 0.001394  eta: <1min   tot: 0h3m59s  (80.7%)3.7%  lr: 0.009610  loss: 0.001450  eta: <1min   tot: 0h3m59s  (80.7%)4.5%  lr: 0.009530  loss: 0.001441  eta: <1min   tot: 0h4m0s  (80.9%)%  lr: 0.009429  loss: 0.001442  eta: <1min   tot: 0h4m0s  (81.1%)6.3%  lr: 0.009379  loss: 0.001435  eta: <1min   tot: 0h4m1s  (81.3%)%  lr: 0.009379  loss: 0.001443  eta: <1min   tot: 0h4m1s  (81.3%)6.8%  lr: 0.009349  loss: 0.001420  eta: <1min   tot: 0h4m1s  (81.4%)7.3%  lr: 0.009279  loss: 0.001418  eta: <1min   tot: 0h4m1s  (81.5%)7.7%  lr: 0.009259  loss: 0.001399  eta: <1min   tot: 0h4m1s  (81.5%)8.1%  lr: 0.009199  loss: 0.

Epoch: 100.0%  lr: 0.000000  loss: 0.001372  eta: <1min   tot: 0h4m55s  (100.0%).6%  lr: 0.002633  loss: 0.001389  eta: <1min   tot: 0h4m38s  (93.9%)70.1%  lr: 0.002613  loss: 0.001387  eta: <1min   tot: 0h4m39s  (94.0%)70.5%  lr: 0.002603  loss: 0.001387  eta: <1min   tot: 0h4m39s  (94.1%)71.0%  lr: 0.002573  loss: 0.001392  eta: <1min   tot: 0h4m39s  (94.2%)72.2%  lr: 0.002402  loss: 0.001395  eta: <1min   tot: 0h4m40s  (94.4%)72.5%  lr: 0.002372  loss: 0.001395  eta: <1min   tot: 0h4m40s  (94.5%)72.8%  lr: 0.002352  loss: 0.001393  eta: <1min   tot: 0h4m40s  (94.6%)72.9%  lr: 0.002352  loss: 0.001394  eta: <1min   tot: 0h4m40s  (94.6%)73.3%  lr: 0.002282  loss: 0.001394  eta: <1min   tot: 0h4m40s  (94.7%)%  lr: 0.002232  loss: 0.001391  eta: <1min   tot: 0h4m41s  (94.7%)74.3%  lr: 0.002172  loss: 0.001390  eta: <1min   tot: 0h4m41s  (94.9%)74.8%  lr: 0.002162  loss: 0.001389  eta: <1min   tot: 0h4m41s  (95.0%)75.2%  lr: 0.002142  loss: 0.001387  eta: <1min   tot: 0h4m41s  (95.0%)76.

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [75]:
starspace_embeddings = {}
for line in open('starspace_embedding.tsv', encoding='utf-8'):
    tokens = line.strip().split('\t')
    starspace_embeddings[tokens[0]] = np.array(tokens[1:],dtype=np.float64)
    ######### YOUR CODE HERE #############

In [77]:
ss_prepared_ranking = []
for line in tqdm(prepared_validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 10000/10000 [02:33<00:00, 65.02it/s]


In [78]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

DCG@   1: 0.520 | Hits@   1: 0.520
DCG@   5: 0.616 | Hits@   5: 0.699
DCG@  10: 0.635 | Hits@  10: 0.757
DCG@ 100: 0.666 | Hits@ 100: 0.906
DCG@ 500: 0.675 | Hits@ 500: 0.981
DCG@1000: 0.678 | Hits@1000: 1.000


Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [79]:
starspace_ranks_results = []
prepared_test_data = 'data/test_prepared.tsv'######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

Current answer for task StarSpaceRanks is: 75	48	96	20	15	78	71	89	37	57	29	49	10	6	99	67	9	74	73	94	65	28	27	83	95	30	66	98	58	4	64	36	62	69	2...


### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [80]:
STUDENT_EMAIL = 'tianqi.terence.liu@gmail.com'# EMAIL 
STUDENT_TOKEN = 'ZVYXt5j9EFxAH3b3'# TOKEN 
grader.status()

You want to submit these parts:
Task Question2Vec: 0.01929389126598835
-0.02872721292078495
0.0460561104118824
0.0852593332529068
0.0243055559694767
-0...
Task HitsCount: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.30000000000000004
0.4
0.5
0....
Task DCGScore: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...
Task W2VTokenizedRanks: 95	94	7	9	64	37	32	93	24	100	98	17	60	6	97	49	70	38	42	96	30	21	2	65	67	45	27	26	57	62	11	88	56	66	7...
Task StarSpaceRanks: 75	48	96	20	15	78	71	89	37	57	29	49	10	6	99	67	9	74	73	94	65	28	27	83	95	30	66	98	58	4	64	36	62	69	2...


If you want to submit these answers, run cell below

In [81]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
